In [1]:
import subprocess
import json
import os
from constants import BTC, BTCTEST, ETH
from pprint import pprint
from bit import PrivateKeyTestnet
from bit.network import NetworkAPI
from web3 import Web3, middleware, Account
from web3.gas_strategies.time_based import medium_gas_price_strategy
from web3.middleware import geth_poa_middleware
from web3.middleware import pythonic_middleware
from eth_account import Account


In [2]:
# connect Web3
w3 = Web3(Web3.HTTPProvider(os.getenv('WEB3_PROVIDER', 'http://localhost:8545')))
# enable PoA middleware
w3.middleware_onion.inject(geth_poa_middleware, layer=0)


In [3]:
w3.eth.setGasPriceStrategy(medium_gas_price_strategy)

In [4]:

mnemonic = os.getenv("MNEMONIC", "blossom foam explain join tree slight host client tumble robust drink smart")


In [5]:
def derive_wallets(coin=BTC, mnemonic=mnemonic, depth=3):
    command = (
        f"./derive -g --mnemonic='{mnemonic}' --coin={coin} --numderive={depth} --format=json",
        stdout=subprocess.PIPE,
        shell=True)
    (output, err) = p.communicate()
    p_status = p.wait()
    return json.loads(output.decode("utf-8"))

In [6]:
def priv_key_to_account(coin, priv_key):
    if coin == ETH:
        return Account.privateKeyToAccount(priv_key)
    if coin == BTCTEST:
        return PrivateKeyTestnet(priv_key)

In [7]:
def create_tx(coin, account, to, amount):
    if coin == ETH:
        value = w3.toWei(amount, "ether") # convert 1.2 ETH to 120000000000 wei
        gasEstimate = w3.eth.estimateGas({ "to": to, "from": account.address, "amount": value })
        return {
            "to": to,
            "from": account.address,
            "value": value,
            "gas": gasEstimate,
            "gasPrice": w3.eth.generateGasPrice(),
            "nonce": w3.eth.getTransactionCount(account.address),
            "chainId": w3.net.chainId
        }
    if coin == BTCTEST:
        return PrivateKeyTestnet.prepare_transaction(account.address, [(to, amount, BTC)])

In [8]:
def send_tx(coin, account, to, amount):
    if coin == ETH:
        raw_tx = create_tx(coin, account, to, amount)
        signed = account.signTransaction(raw_tx)
        return w3.eth.sendRawTransaction(signed.rawTransaction)
    if coin == BTCTEST:
        raw_tx = create_tx(coin, account, to, amount)
        signed = account.sign_transaction(raw_tx)
        return NetworkAPI.broadcast_tx_testnet(signed)


In [16]:
coins = {'ETH': derive_wallets(coin=ETH, mnemonic=mnemonic, depth=3),'BTCTEST': derive_wallets(coin=BTCTST, mnemonic=mnemonic, depth=3),
}
pprint(coins)


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [11]:

account = priv_key_to_account(BTCTEST, coins[BTCTEST][0]['privkey'])

NameError: name 'coins' is not defined